In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:1]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ocular-disease-recognition-odir5k/full_df.csv
/kaggle/input/ocular-disease-recognition-odir5k/preprocessed_images/3419_left.jpg
/kaggle/input/ocular-disease-recognition-odir5k/ODIR-5K/ODIR-5K/data.xlsx
/kaggle/input/ocular-disease-recognition-odir5k/ODIR-5K/ODIR-5K/Testing Images/3552_right.jpg
/kaggle/input/ocular-disease-recognition-odir5k/ODIR-5K/ODIR-5K/Training Images/3419_left.jpg
/kaggle/input/ocular-data-all-imgs/Combin_tr_te/3419_left.jpg
/kaggle/input/ocular-final-csv/Complete_dataset_forML.csv


In [2]:
!pip install tf_clahe --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.21.0, which is not installed.
dask-cudf 21.10.1 requires cupy-cuda114, which is not installed.
beatrix-jupyterlab 3.1.7 requires google-cloud-bigquery-storage, which is not installed.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.19.5 which is incompatible.
tfx-bsl 1.9.0 requires tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,!=2.8.*,<3,>=1.15.5, but you have tensorflow 2.6.4 which is incompatible.
tensorflow-transform 1.9.0 requires tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,!=2.8.*,<2.10,>=1.15.5, but you have tensorflow 2.6.4 which is incompatible.
tensorflow-serving-api 2.9.0 requires tensorflow<3,>=2.9.0, but you have tensorflow 2.6.4 which is incompatible.
rich 12.6.0 req

In [3]:
import pandas as pd
import numpy as np
import ast
import cv2
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split 

from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras import layers,Model
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
import tensorflow as tf
import tensorflow_datasets as tfds
from tqdm import tqdm
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,Dense
from sklearn.model_selection import KFold, StratifiedKFold

from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE,RandomOverSampler
from imblearn.under_sampling import NearMiss, RandomUnderSampler

import tf_clahe

# Helper Functions and things


In [4]:
def fast_clahe(img):
    return tf_clahe.clahe(img, gpu_optimized=True)
def get_unique_count(col):
    values,counts=np.unique(col,return_counts=True)
    for i,j,k in list(zip(values,counts,counts/sum(counts))):
        print(i,j,round(k,2)) 
@tf.function(experimental_compile=True)  # Enable XLA
def fast_clahe(img):
    return tf_clahe.clahe(img, gpu_optimized=True)

In [5]:
def create_dataset(image_names,labels,clahe=None):
    images=[]
    classes=[]
    for img,label in tqdm(list(zip(image_names,labels))):
        image_path = CFG.images_path+'/'+img
#         try:
        image = cv2.imread(image_path)
        #plt.imshow(image)
        mask = image > 0
        
        # Coordinates of coloured pixels.
        coordinates = np.argwhere(mask)
        # Binding box of non-black pixels.
        x0, y0, s0 = coordinates.min(axis=0)
        x1, y1, s1 = coordinates.max(axis=0) + 1  # slices are exclusive at the top
        #print(x0)
        
        # Get the contents of the bounding box.
        cropped = image[x0:x1, y0:y1]

        #img = tf.image.resize(cropped,CFG.img_size,antialias=True)
        image = cv2.resize(cropped,(CFG.image_size,CFG.image_size))
        #image = cv2.resize(image,(CFG.image_size,CFG.image_size))
#         except:
#             continue

        if clahe:
                try:
                    image = fast_clahe(image)
                    #mage=np.array(image)
                except:
                    print(image)
        dummy=[0]*4
        dummy[label]=1
        images.append(image)
        classes.append(dummy)
    return tf.data.Dataset.from_tensor_slices((images,classes))

In [6]:
class CFG:
    csv_path='/kaggle/input/ocular-final-csv/Complete_dataset_forML.csv'
    images_path='/kaggle/input/ocular-data-all-imgs/Combin_tr_te'
    image_size=512
    n_classes=4

In [7]:
from tensorflow.keras.layers.experimental import preprocessing
trainAug = Sequential([
    #preprocessing.Rescaling(scale=1.0 / 255),
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomFlip("vertical"),
    #preprocessing.RandomRotation(0.9)
])

2022-12-12 16:06:21.777029: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-12 16:06:21.870391: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-12 16:06:21.871152: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-12 16:06:21.872820: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [8]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
def get_metrics(testDS,test_y,model):
    y_preds=model.predict(testDS)
    y_preds_class=[]
    for i in y_preds:
        y_preds_class.append(np.argmax(i))
    y_test_class=[]
    for i in test_y:
        y_test_class.append(np.argmax(i))
    print(classification_report(y_test_class,y_preds_class))

In [ ]:
df=pd.read_csv(CFG.csv_path)
# df['Label']= df['Label']-1
# df=df[(df['Label']==0) | (df['Label']==2) | (df['Label']==3) | (df['Label']==5)]
df.head()

In [ ]:
get_unique_count(df['Label'])

0-Normal
1- Glacuma
2-cataract
3- Hypertension

In [13]:
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE,RandomOverSampler
from imblearn.under_sampling import NearMiss, RandomUnderSampler

def sampling_technique(df,test_condition):
    #594 500 397 203
    if test_condition=='Under-Sampling':
        count_class_0 = 300
        count_class_1 = 300
        count_class_2 = 300
        count_class_3 = 300
        
        pipe = make_pipeline(
        RandomUnderSampler(sampling_strategy={0: count_class_0}),
        RandomOverSampler(sampling_strategy={1: count_class_1}),
        RandomOverSampler(sampling_strategy={2: count_class_2}),
        RandomOverSampler(sampling_strategy={3: count_class_3})
    #     RandomOverSampler(sampling_strategy={4: count_class_4}),
    #     RandomOverSampler(sampling_strategy={5: count_class_5}),
    #     RandomOverSampler(sampling_strategy={6: count_class_6}),
    #     RandomOverSampler(sampling_strategy={7: count_class_7})
        )

    if test_condition=='No-Sampling':
        count_class_0 = 1000
        count_class_1 = len(df[df['Label']==1])
        count_class_2 = len(df[df['Label']==2])
        count_class_3 = len(df[df['Label']==3])
        
        pipe = make_pipeline(
        RandomUnderSampler(sampling_strategy={0: count_class_0}),
        RandomUnderSampler(sampling_strategy={1: count_class_1}),
        RandomOverSampler(sampling_strategy={2: count_class_2}),
        RandomOverSampler(sampling_strategy={3: count_class_3})
    #     RandomOverSampler(sampling_strategy={4: count_class_4}),
    #     RandomOverSampler(sampling_strategy={5: count_class_5}),
    #     RandomOverSampler(sampling_strategy={6: count_class_6}),
    #     RandomOverSampler(sampling_strategy={7: count_class_7})
        )

    if test_condition=='Over-Sampling':
        count_class_0 = 700
        count_class_1 = 600
        count_class_2 = 600
        count_class_3 = 600

        pipe = make_pipeline(
            RandomUnderSampler(sampling_strategy={0: count_class_0}),
            RandomOverSampler(sampling_strategy={1: count_class_1}),
            RandomOverSampler(sampling_strategy={2: count_class_2}),
            RandomOverSampler(sampling_strategy={3: count_class_3})
        #     RandomOverSampler(sampling_strategy={4: count_class_4}),
        #     RandomOverSampler(sampling_strategy={5: count_class_5}),
        #     RandomOverSampler(sampling_strategy={6: count_class_6}),
        #     RandomOverSampler(sampling_strategy={7: count_class_7})
        )
    X_smt,y_smt = pipe.fit_resample(df, df['Label'])
    return X_smt

In [12]:
df=sampling_technique(df,'No-Sampling')

In [13]:
get_unique_count(df['Label'])

0 1000 0.55
1 313 0.17
2 326 0.18
3 193 0.11


# Getting Train, Validation and Test

In [14]:

train, test = train_test_split(df, test_size=0.10, stratify=df['Label'],random_state=1) 
train,val= train_test_split(train, test_size=0.10, random_state=1,stratify=train['Label'])

print("train",len(train))
print("test",len(test))
print("Val",len(val))


train 1483
test 184
Val 165


In [15]:
get_unique_count(train['Label'])

0 810 0.55
1 253 0.17
2 264 0.18
3 156 0.11


In [16]:
train.to_csv('train.csv',index=False)
test.to_csv('test.csv',index=False)
val.to_csv('val.csv',index=False)


In [17]:
#train=X_smt

In [18]:
print(get_unique_count(train['Label'].values))
print(get_unique_count(val['Label'].values))
print(get_unique_count(test['Label'].values))

0 810 0.55
1 253 0.17
2 264 0.18
3 156 0.11
None
0 90 0.55
1 28 0.17
2 29 0.18
3 18 0.11
None
0 100 0.54
1 32 0.17
2 33 0.18
3 19 0.1
None


In [19]:
# train_x,train_y = create_dataset(train['File'].values,train['Label'].values)
# val_x,val_y = create_dataset(val['File'].values,val['Label'].values)
# test_x,test_y = create_dataset(test['File'].values,test['Label'].values)

In [20]:
trainDS = create_dataset(train['File'].values,train['Label'].values)


100%|██████████| 1483/1483 [07:24<00:00,  3.34it/s]
2022-12-12 09:40:49.382616: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 9330229248 exceeds 10% of free system memory.
2022-12-12 09:48:29.786354: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1166278656 exceeds 10% of free system memory.
2022-12-12 09:48:32.521846: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1166278656 exceeds 10% of free system memory.


In [21]:
valDS = create_dataset(val['File'].values,val['Label'].values)
testDS = create_dataset(test['File'].values,test['Label'].values)

100%|██████████| 165/165 [00:42<00:00,  3.89it/s]
2022-12-12 09:49:15.933735: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1038090240 exceeds 10% of free system memory.
100%|██████████| 184/184 [00:47<00:00,  3.89it/s]
2022-12-12 09:50:54.059805: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1157627904 exceeds 10% of free system memory.


In [22]:
# print("train_img_X: ",np.array(train_x).shape)
# print("test_img_X: ",np.array(test_x).shape)
# print("train_labels: ",np.array(train_y).shape)
# print("test_labels: ",np.array(test_y).shape)

In [23]:
#np.array(train_x).shape

In [24]:
# trainDS = tf.data.Dataset.from_tensor_slices((train_x,train_y))
# del train_x,train_y
# valDS = tf.data.Dataset.from_tensor_slices((val_x,val_y))
# del val_x,val_y
# testDS = tf.data.Dataset.from_tensor_slices((test_x,test_y))

In [25]:
tf.data.experimental.save(trainDS,'trainDS')


2022-12-12 09:51:51.698842: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [26]:
tf.data.experimental.save(valDS,'ValDS')
tf.data.experimental.save(testDS,'TestDS')

In [27]:
trainDS=trainDS.batch(16).prefetch(tf.data.AUTOTUNE)
valDS=valDS.batch(16).prefetch(tf.data.AUTOTUNE)
testDS=testDS.batch(16).prefetch(tf.data.AUTOTUNE)

# Model Training

In [ ]:
checkpoint = ModelCheckpoint('name.h5',monitor="val_acc",verbose=1,save_best_only=True,
                             save_weights_only=False,period=1)

## Different models

In [10]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications import EfficientNetB0,EfficientNetB7
from tensorflow.keras.applications.xception import Xception

In [9]:
def baseline_model(model):
        vgg = model
        vgg.trainable=False
        input_layer = layers.Input(shape=(CFG.image_size,CFG.image_size,3), name='input') 
        
        ptm = vgg(input_layer)

        l1 = layers.GlobalAveragePooling2D(name='global_average_pooling2d')(ptm)
        l2 = layers.BatchNormalization()(l1)       
        l3 = layers.Dense(128,activation='relu')(l2)

#         l4 = layers.BatchNormalization()(l3)       
#         l5 = layers.Dense(128,activation='relu')(l4)

        
        output = layers.Dense(units=CFG.n_classes, 
                              activation='softmax', 
                              name='output')(l3)
        model = Model (input_layer, output)    
        model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0002), 
                      loss='categorical_crossentropy', 
                      metrics=[ 'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')])
        return model

# efficient_model=baseline_model()
# efficient_model.summary()

In [30]:
vgg = baseline_model(VGG19(weights="imagenet",include_top = False,input_shape=(CFG.image_size,CFG.image_size,3)))

80150528/80134624 [==============================] - 0s 0us/step


/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [31]:
vgg.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 512, 512, 3)]     0         
_________________________________________________________________
vgg19 (Functional)           (None, 16, 16, 512)       20024384  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dense (Dense)                (None, 128)               65664     
_________________________________________________________________
output (Dense)               (None, 4)                 516       
Total params: 20,092,612
Trainable params: 67,204
Non-trainable params: 20,025,408
____________________________________________

In [32]:
# model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy",tf.keras.metrics.Precision(name='precision'),
#         tf.keras.metrics.Recall(name='recall')])

In [33]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
checkpoint = ModelCheckpoint('name.h5',monitor="val_acc",verbose=1,save_best_only=True,
                             save_weights_only=False,period=1)

In [34]:
vgg.fit(trainDS,batch_size=16,epochs=15,validation_data=(valDS),
                    verbose=1,callbacks=[checkpoint])

Epoch 1/15


2022-12-12 09:52:04.252818: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


93/93 [==============================] - 38s 263ms/step - loss: 1.1035 - accuracy: 0.5570 - precision: 0.6592 - recall: 0.4343 - val_loss: 0.9236 - val_accuracy: 0.6727 - val_precision: 0.6887 - val_recall: 0.4424
Epoch 2/15
93/93 [==============================] - 17s 183ms/step - loss: 0.7142 - accuracy: 0.7370 - precision: 0.7776 - recall: 0.6649 - val_loss: 0.7191 - val_accuracy: 0.7576 - val_precision: 0.7937 - val_recall: 0.6061
Epoch 3/15
93/93 [==============================] - 17s 183ms/step - loss: 0.5934 - accuracy: 0.7775 - precision: 0.8190 - recall: 0.7262 - val_loss: 0.6134 - val_accuracy: 0.8061 - val_precision: 0.8382 - val_recall: 0.6909
Epoch 4/15
93/93 [==============================] - 17s 183ms/step - loss: 0.5116 - accuracy: 0.8119 - precision: 0.8501 - recall: 0.7687 - val_loss: 0.5514 - val_accuracy: 0.8424 - val_precision: 0.8777 - val_recall: 0.7394
Epoch 5/15
93/93 [==============================] - 17s 184ms/step - loss: 0.4493 - accuracy: 0.8429 - precisio

In [35]:
test_y = np.concatenate([y for _, y in testDS], axis=0)

#### VGG19

In [36]:
get_metrics(testDS,test_y,vgg)

              precision    recall  f1-score   support

           0       0.78      0.89      0.83       100
           1       0.79      0.94      0.86        32
           2       0.62      0.45      0.53        33
           3       0.88      0.37      0.52        19

    accuracy                           0.77       184
   macro avg       0.77      0.66      0.68       184
weighted avg       0.76      0.77      0.75       184



In [37]:
# data= model.evaluate(testDS)
# print(data)
del vgg

In [38]:
#for efficient net
b0=tf.keras.applications.EfficientNetB0(include_top=False, weights='imagenet',input_shape=(CFG.image_size,CFG.image_size,3))
#b7=tf.keras.applications.EfficientNetB7(include_top=False, weights='imagenet',input_shape=(CFG.image_size,CFG.image_size,3))
b4=tf.keras.applications.EfficientNetB4(include_top=False, weights='imagenet',input_shape=(CFG.image_size,CFG.image_size,3))
xception=tf.keras.applications.xception.Xception(include_top=False, weights='imagenet',input_shape=(CFG.image_size,CFG.image_size,3))


83697664/83683744 [==============================] - 0s 0us/step


#### EfficientnetB0

In [39]:
effb0=baseline_model(b0)
effb0.fit(trainDS,batch_size=16,epochs=15,validation_data=(valDS),
                    verbose=1,callbacks=[checkpoint])

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/15
93/93 [==============================] - 16s 116ms/step - loss: 1.0397 - accuracy: 0.5981 - precision: 0.6481 - recall: 0.5105 - val_loss: 1.0051 - val_accuracy: 0.7879 - val_precision: 0.8235 - val_recall: 0.0848
Epoch 2/15
93/93 [==============================] - 9s 98ms/step - loss: 0.6881 - accuracy: 0.7330 - precision: 0.7717 - recall: 0.6837 - val_loss: 0.8226 - val_accuracy: 0.7818 - val_precision: 0.8765 - val_recall: 0.4303
Epoch 3/15
93/93 [==============================] - 9s 97ms/step - loss: 0.6229 - accuracy: 0.7653 - precision: 0.7961 - recall: 0.7215 - val_loss: 0.6762 - val_accuracy: 0.8364 - val_precision: 0.8957 - val_recall: 0.6242
Epoch 4/15
93/93 [==============================] - 9s 96ms/step - loss: 0.5509 - accuracy: 0.7889 - precision: 0.8189 - recall: 0.7498 - val_loss: 0.5809 - val_accuracy: 0.8121 - val_precision: 0.8828 - val_recall: 0.6848
Epoch 5/15
93/93 [==============================] - 9s 97ms/step - loss: 0.5109 - accuracy: 0.8105 - preci

In [40]:
get_metrics(testDS,test_y,effb0)# Get mertic table for B0

              precision    recall  f1-score   support

           0       0.81      0.87      0.84       100
           1       0.77      0.94      0.85        32
           2       0.74      0.61      0.67        33
           3       0.91      0.53      0.67        19

    accuracy                           0.80       184
   macro avg       0.81      0.73      0.75       184
weighted avg       0.80      0.80      0.79       184



In [41]:
del effb0

#### EfficientnetB4

In [42]:
effb4=baseline_model(b4)
effb4.fit(trainDS,batch_size=16,epochs=15,validation_data=(valDS),
                    verbose=1,callbacks=[checkpoint])

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/15
93/93 [==============================] - 35s 273ms/step - loss: 0.9551 - accuracy: 0.6325 - precision: 0.6809 - recall: 0.5698 - val_loss: 0.8860 - val_accuracy: 0.7152 - val_precision: 0.8182 - val_recall: 0.4909
Epoch 2/15
93/93 [==============================] - 22s 240ms/step - loss: 0.6411 - accuracy: 0.7566 - precision: 0.7850 - recall: 0.7114 - val_loss: 0.7579 - val_accuracy: 0.7515 - val_precision: 0.8621 - val_recall: 0.6061
Epoch 3/15
93/93 [==============================] - 22s 241ms/step - loss: 0.5690 - accuracy: 0.7802 - precision: 0.8105 - recall: 0.7471 - val_loss: 0.6692 - val_accuracy: 0.7515 - val_precision: 0.8397 - val_recall: 0.6667
Epoch 4/15
93/93 [==============================] - 22s 240ms/step - loss: 0.5002 - accuracy: 0.8058 - precision: 0.8316 - recall: 0.7728 - val_loss: 0.6042 - val_accuracy: 0.7636 - val_precision: 0.8207 - val_recall: 0.7212
Epoch 5/15
93/93 [==============================] - 22s 239ms/step - loss: 0.4800 - accuracy: 0.8173

In [43]:
get_metrics(testDS,test_y,effb4)

              precision    recall  f1-score   support

           0       0.81      0.88      0.85       100
           1       0.79      0.84      0.82        32
           2       0.71      0.67      0.69        33
           3       0.82      0.47      0.60        19

    accuracy                           0.79       184
   macro avg       0.78      0.72      0.74       184
weighted avg       0.79      0.79      0.79       184



In [44]:
del effb4

#### EfficientnetB7

In [45]:
b7=tf.keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet',input_shape=(CFG.image_size,CFG.image_size,3))

258088960/258076736 [==============================] - 1s 0us/step


In [46]:
effb7=baseline_model(b7)
effb7.fit(trainDS,batch_size=16,epochs=15,validation_data=(valDS),
                    verbose=1,callbacks=[checkpoint])

In [47]:
# get_metrics(testDS,test_y,effb7)# Get mertic table for B7

#### Exception

In [48]:
xception=baseline_model(xception)
xception.fit(trainDS,batch_size=16,epochs=15,validation_data=(valDS),
                    verbose=1,callbacks=[checkpoint])

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/15
93/93 [==============================] - 23s 218ms/step - loss: 1.1785 - accuracy: 0.5367 - precision: 0.6105 - recall: 0.3837 - val_loss: 0.9794 - val_accuracy: 0.6121 - val_precision: 0.6720 - val_recall: 0.5091
Epoch 2/15
93/93 [==============================] - 19s 205ms/step - loss: 0.8123 - accuracy: 0.6932 - precision: 0.7481 - recall: 0.5968 - val_loss: 0.8600 - val_accuracy: 0.6667 - val_precision: 0.7087 - val_recall: 0.5455
Epoch 3/15
93/93 [==============================] - 19s 204ms/step - loss: 0.6741 - accuracy: 0.7444 - precision: 0.8039 - recall: 0.6662 - val_loss: 0.8399 - val_accuracy: 0.6667 - val_precision: 0.7015 - val_recall: 0.5697
Epoch 4/15
93/93 [==============================] - 19s 204ms/step - loss: 0.5739 - accuracy: 0.7923 - precision: 0.8452 - recall: 0.7181 - val_loss: 0.8394 - val_accuracy: 0.6606 - val_precision: 0.6940 - val_recall: 0.5636
Epoch 5/15
93/93 [==============================] - 19s 205ms/step - loss: 0.4959 - accuracy: 0.8254

In [49]:
get_metrics(testDS,test_y,xception)

              precision    recall  f1-score   support

           0       0.66      0.82      0.73       100
           1       0.65      0.75      0.70        32
           2       0.56      0.27      0.37        33
           3       0.43      0.16      0.23        19

    accuracy                           0.64       184
   macro avg       0.58      0.50      0.51       184
weighted avg       0.62      0.64      0.61       184



# Augmentations

In [60]:
import gc

In [55]:
del trainDS
del valDS
del testDS
#del train_x

In [56]:

trainDS = create_dataset(train['File'].values,train['Label'].values,clahe=True)
valDS = create_dataset(val['File'].values,val['Label'].values,clahe=True)
testDS = create_dataset(test['File'].values,test['Label'].values,clahe=True)

100%|██████████| 184/184 [00:50<00:00,  3.66it/s]


In [57]:
# trainDS = tf.data.Dataset.from_tensor_slices((train_x,train_y))
# valDS = tf.data.Dataset.from_tensor_slices((val_x,val_y))
# testDS = tf.data.Dataset.from_tensor_slices((test_x,test_y))

In [58]:
# trainDS=trainDS.batch(16).prefetch(tf.data.AUTOTUNE)
valDS=valDS.batch(16).prefetch(tf.data.AUTOTUNE)
testDS=testDS.batch(16).prefetch(tf.data.AUTOTUNE)

In [54]:
#plt.imshow(np.array(train_x)[0])

NameError: name 'train_x' is not defined

In [ ]:
#plt.imshow(np.array(train_x)[0])

In [60]:
trainDS_noaug=trainDS.batch(16).prefetch(tf.data.AUTOTUNE)
trainDSAug = (
    trainDS.batch(16)
    .map(lambda x, y: (trainAug(x), y))
)

#### No augmentations- only clahe

In [61]:
b4=tf.keras.applications.EfficientNetB4(include_top=False, weights='imagenet',input_shape=(CFG.image_size,CFG.image_size,3))
effb4=baseline_model(b4)
effb4.fit(trainDS_noaug,batch_size=16,epochs=15,validation_data=(valDS),
                    verbose=1)

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/15
93/93 [==============================] - 35s 268ms/step - loss: 0.9285 - accuracy: 0.6379 - precision: 0.6881 - recall: 0.5772 - val_loss: 0.8649 - val_accuracy: 0.7273 - val_precision: 0.8941 - val_recall: 0.4606
Epoch 2/15
93/93 [==============================] - 22s 241ms/step - loss: 0.6277 - accuracy: 0.7478 - precision: 0.7807 - recall: 0.7080 - val_loss: 0.7551 - val_accuracy: 0.7879 - val_precision: 0.8687 - val_recall: 0.5212
Epoch 3/15
93/93 [==============================] - 23s 243ms/step - loss: 0.5431 - accuracy: 0.7889 - precision: 0.8232 - recall: 0.7566 - val_loss: 0.6230 - val_accuracy: 0.7818 - val_precision: 0.8593 - val_recall: 0.7030
Epoch 4/15
93/93 [==============================] - 22s 241ms/step - loss: 0.5004 - accuracy: 0.8051 - precision: 0.8372 - recall: 0.7802 - val_loss: 0.5741 - val_accuracy: 0.7879 - val_precision: 0.8133 - val_recall: 0.7394
Epoch 5/15
93/93 [==============================] - 22s 241ms/step - loss: 0.4373 - accuracy: 0.8449

In [62]:
test_y = np.concatenate([y for _, y in testDS], axis=0)

In [63]:
get_metrics(testDS,test_y,effb4)# Get mertic table for B0

              precision    recall  f1-score   support

           0       0.79      0.87      0.83       100
           1       0.82      0.84      0.83        32
           2       0.66      0.58      0.61        33
           3       0.83      0.53      0.65        19

    accuracy                           0.78       184
   macro avg       0.77      0.70      0.73       184
weighted avg       0.78      0.78      0.77       184



#### With augmentations

In [64]:
effb4_aug=baseline_model(b4)
effb4_aug.fit(trainDSAug,batch_size=16,epochs=15,validation_data=(valDS),
                    verbose=1,callbacks=[checkpoint])

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/15
93/93 [==============================] - 49s 418ms/step - loss: 0.9586 - accuracy: 0.6332 - precision: 0.6874 - recall: 0.5651 - val_loss: 0.8566 - val_accuracy: 0.7152 - val_precision: 0.8229 - val_recall: 0.4788
Epoch 2/15
93/93 [==============================] - 32s 346ms/step - loss: 0.6523 - accuracy: 0.7417 - precision: 0.7755 - recall: 0.7013 - val_loss: 0.7405 - val_accuracy: 0.7576 - val_precision: 0.8598 - val_recall: 0.5576
Epoch 3/15
93/93 [==============================] - 33s 350ms/step - loss: 0.5839 - accuracy: 0.7788 - precision: 0.8109 - recall: 0.7431 - val_loss: 0.6357 - val_accuracy: 0.7697 - val_precision: 0.8321 - val_recall: 0.6606
Epoch 4/15
93/93 [==============================] - 32s 349ms/step - loss: 0.5438 - accuracy: 0.7802 - precision: 0.8178 - recall: 0.7505 - val_loss: 0.5914 - val_accuracy: 0.7455 - val_precision: 0.8333 - val_recall: 0.7273
Epoch 5/15
93/93 [==============================] - 32s 348ms/step - loss: 0.4951 - accuracy: 0.8085

In [65]:
get_metrics(testDS,test_y,effb4_aug)# Get mertic table for B0

              precision    recall  f1-score   support

           0       0.79      0.87      0.83       100
           1       0.81      0.78      0.79        32
           2       0.71      0.67      0.69        33
           3       0.75      0.47      0.58        19

    accuracy                           0.78       184
   macro avg       0.76      0.70      0.72       184
weighted avg       0.77      0.78      0.77       184



In [63]:
train=sampling_technique(train,'Over-Sampling')

# OverSampling

In [16]:
from tensorflow.keras.layers.experimental import preprocessing
trainAug = Sequential([
    #preprocessing.Rescaling(scale=1.0 / 255),
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomFlip("vertical"),
    #preprocessing.RandomRotation(0.9)
])

In [17]:
df=sampling_technique(df,'No-Sampling')

In [18]:
train, test = train_test_split(df, test_size=0.10, stratify=df['Label'],random_state=1) 
train,val= train_test_split(train, test_size=0.10, random_state=1,stratify=train['Label'])

print("train",len(train))
print("test",len(test))
print("Val",len(val))


train 1483
test 184
Val 165


In [19]:
# train=pd.read_csv('/kaggle/working/train.csv')
# val=pd.read_csv('/kaggle/working/val.csv')
# test=pd.read_csv('/kaggle/working/test.csv')

In [20]:
train_up=sampling_technique(train,'Under-Sampling')

In [21]:
get_unique_count(train_up['Label'])

0 300 0.25
1 300 0.25
2 300 0.25
3 300 0.25


In [22]:
get_unique_count(val['Label'])

0 90 0.55
1 28 0.17
2 29 0.18
3 18 0.11


In [23]:
trainDS= create_dataset(train_up['File'].values,train_up['Label'].values)
valDS = create_dataset(val['File'].values,val['Label'].values)
testDS = create_dataset(test['File'].values,test['Label'].values)

100%|██████████| 1200/1200 [05:53<00:00,  3.40it/s]
2022-12-12 12:21:21.862170: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 7549747200 exceeds 10% of free system memory.
100%|██████████| 184/184 [00:49<00:00,  3.73it/s]


In [ ]:
# trainDS = tf.data.Dataset.from_tensor_slices((train_x,train_y))
# valDS = tf.data.Dataset.from_tensor_slices((val_x,val_y))
# testDS = tf.data.Dataset.from_tensor_slices((test_x,test_y))

In [24]:
trainDSAug = (
    trainDS.batch(16)
    .map(lambda x, y: (trainAug(x), y))
)

In [25]:
trainDSAug=trainDS.batch(16).map(lambda x, y: (trainAug(x), y)).prefetch(tf.data.AUTOTUNE)
valDS=valDS.batch(16).prefetch(tf.data.AUTOTUNE)
testDS=testDS.batch(16).prefetch(tf.data.AUTOTUNE)

In [26]:
b0=tf.keras.applications.EfficientNetB4(include_top=False, weights='imagenet',input_shape=(CFG.image_size,CFG.image_size,3))
effb0=baseline_model(b0)
effb0.fit(trainDSAug,batch_size=16,epochs=15,validation_data=(valDS),
                    verbose=1)


71696384/71686520 [==============================] - 0s 0us/step


/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
2022-12-12 12:33:55.983483: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/15


2022-12-12 12:34:07.658541: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


75/75 [==============================] - 39s 296ms/step - loss: 1.8171 - accuracy: 0.3517 - precision: 0.3800 - recall: 0.2558 - val_loss: 1.5627 - val_accuracy: 0.1758 - val_precision: 1.0000 - val_recall: 0.0061
Epoch 2/15
75/75 [==============================] - 19s 254ms/step - loss: 1.2139 - accuracy: 0.4933 - precision: 0.6009 - recall: 0.3425 - val_loss: 1.5924 - val_accuracy: 0.2121 - val_precision: 0.8000 - val_recall: 0.0242
Epoch 3/15
75/75 [==============================] - 19s 258ms/step - loss: 0.9940 - accuracy: 0.5725 - precision: 0.6942 - recall: 0.4483 - val_loss: 1.5909 - val_accuracy: 0.2606 - val_precision: 0.4286 - val_recall: 0.0727
Epoch 4/15
75/75 [==============================] - 20s 260ms/step - loss: 0.8609 - accuracy: 0.6425 - precision: 0.7317 - recall: 0.5092 - val_loss: 1.6566 - val_accuracy: 0.2667 - val_precision: 0.3729 - val_recall: 0.1333
Epoch 5/15
75/75 [==============================] - 20s 263ms/step - loss: 0.7304 - accuracy: 0.7050 - precisio

In [27]:
test_y = np.concatenate([y for _, y in testDS], axis=0)

In [28]:
get_metrics(testDS,test_y,effb0)

              precision    recall  f1-score   support

           0       0.77      0.10      0.18       100
           1       0.93      0.41      0.57        32
           2       0.21      0.76      0.33        33
           3       0.08      0.16      0.11        19

    accuracy                           0.28       184
   macro avg       0.50      0.36      0.29       184
weighted avg       0.63      0.28      0.26       184



# K-fold ensemble

In [24]:
CFG.folds=4

In [25]:
df=pd.read_csv(CFG.csv_path)
df=sampling_technique(df,'No-Sampling')

In [26]:
train, test = train_test_split(df, test_size=0.10, stratify=df['Label'],random_state=1) 
train['k-fold']=-1
train=train.reset_index()

In [27]:
get_unique_count(train['Label'])

0 900 0.55
1 281 0.17
2 293 0.18
3 174 0.11


In [28]:
len(test)

184

In [30]:
for i,(train_index, test_index) in enumerate(StratifiedKFold(n_splits=CFG.folds,shuffle=False).split(train,train['Label'])):
    train.loc[test_index,'k-fold']=i+1

In [31]:
get_unique_count(train['k-fold'])

1 412 0.25
2 412 0.25
3 412 0.25
4 412 0.25


In [32]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
testDS = create_dataset(test['File'].values,test['Label'].values)
testDS=testDS.batch(16).prefetch(tf.data.AUTOTUNE)

100%|██████████| 184/184 [00:52<00:00,  3.48it/s]


In [ ]:
i=1
train_d=train[(train['k-fold']!=i)]
train_v=train[(train['k-fold']==i)]
print(len(train_d))
print(len(train_v))
trainDS = create_dataset(train_d['File'].values,train_d['Label'].values)
valDS = create_dataset(train_v['File'].values,train_v['Label'].values)

trainDS=trainDS.batch(16).prefetch(tf.data.AUTOTUNE)
valDS=valDS.batch(16).prefetch(tf.data.AUTOTUNE)
print('Starting Train for fold: '+str(i))
#     trainDS = tf.data.Dataset.from_tensor_slices((train_X,train_Y))
#     valDS = tf.data.Dataset.from_tensor_slices((val_x,val_y))
model_name='efficient_net_fold_'+str(i)
checkpoint = ModelCheckpoint(model_name+'.h5',monitor="val_accuracy",verbose=1,save_best_only=True,
                         save_weights_only=False,period=1)
b0=tf.keras.applications.EfficientNetB4(include_top=False, weights='imagenet',input_shape=(CFG.image_size,CFG.image_size,3))
effb0=baseline_model(b0)
effb0.fit(trainDS,batch_size=16,epochs=15,validation_data=(valDS),
                    verbose=1,callbacks=[checkpoint])

1236
412


100%|██████████| 1236/1236 [05:54<00:00,  3.49it/s]